In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Dropout, Add, LSTM, ELU, Dropout, MaxPooling2D, LSTM
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, RMSprop
from tensorflow.keras import regularizers

import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tf.__version__

'2.13.0'

# Data Pre-Processing

Open **play_style_train.csv** file and split the games into a list.
Every row of csv: `PSL0000000001,1,B[pd],W[dp],B[qp],W[dc],B[nq],W[nc],B[qf],W[kd],B[ce],W[dg],B[dd],W[cc],B[fd],W[ed],B[ee],W[ec],B[ge],W[gc],B[di]`.

Columns are:

    1. PSL0000000001: Game ID
    2. 1: Game Style
    3-... : Moves, the last move represents the play style (B[di] in this case)
    
We cropped only the moves to game list as:

In [4]:
df = open('/content/drive/Shareddrives/AICUP圍棋Dataset/play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]
print(game_styles)

[1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 3, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 2, 3, 3, 3, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 3, 3, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 3, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 

Create a dictionary to convert the coordinates from characters to numbers

In [5]:
mapping_BW = {'B':0,'W':1}
chars = 'abcdefghijklmnopqrs'
mapping_letter = {k:v for v,k in enumerate(chars)}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 2 dimensional feature map to represent the data as below:
 1. Occupied areas: mark them as 1 and the empty places as 0
 2. The last move in the table: mark the position of the last move as 1 and the rest as 0

The target is to predict the game style (1, 2 or 3) from the state of the game table. Later this will be one-hot encoded.

In [6]:
def prepare_input(moves):
    x = np.zeros((19, 19, 10))
    for move in moves:
        color = move.split('[')[0]
        x_coord = re.search(r'\[([^\]]+)\]', move).group(1)[0]
        y_coord = re.search(r'\[([^\]]+)\]', move).group(1)[1]

        bw = mapping_BW[color]
        column = mapping_letter[x_coord]
        row = mapping_letter[y_coord]
        x[row,column,0] = 1
    if moves:
        last_move_column = mapping_letter[moves[-1][2]]
        last_move_row = mapping_letter[moves[-1][3]]
        x[row,column,1] = 1
    return x

In [7]:
# Check how many samples can be obtained
n_games = 0
for game in games:
    n_games += 1
print(f"Total Games: {n_games}")

Total Games: 26615


Since play style training has smaller dataset comparing to kyu or dan training, we can put the complete dataset to memory. Still, it is better to create a data generator.

In [8]:
import re

mapping_BW = {'B':0,'W':1}
chars = 'abcdefghijklmnopqrs'
mapping_letter = {k:v for v,k in enumerate(chars)}

x = []
for game in games:
  train = []
  moves = game.split(',')
  x.append(prepare_input(moves))

x = np.array(x)

In [9]:
print(x.shape)

(26615, 19, 19, 10)


In [10]:
y = np.array(game_styles)-1
y.shape

(26615,)

In [11]:
np.bincount(y)

array([8184, 9403, 9028])

Target is one-hot encoded and loss is changed to `categorical_crossentropy`

In [12]:
y_hot = tf.one_hot(y, depth=3)
print(y_hot)

tf.Tensor(
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]], shape=(26615, 3), dtype=float32)


Dataset splitting: 90% Training, 10% validation

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x, y_hot.numpy(), test_size=0.10)

In [14]:
x_train

array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

# Training

### Simple DCNN Model:

In [28]:
def create_model():
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(19, 19, 10)))
    model.add(BatchNormalization())

    for i in range(10):
      model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu', kernel_regularizer = regularizers.l2(0.0001),use_bias=True))
      model.add(BatchNormalization())


    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Flatten())


    model.add(Dense(10, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.add(Dropout(0.05))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [29]:
model = create_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 19, 19, 16)        1456      
                                                                 
 batch_normalization_33 (Ba  (None, 19, 19, 16)        64        
 tchNormalization)                                               
                                                                 
 conv2d_34 (Conv2D)          (None, 19, 19, 16)        2320      
                                                                 
 batch_normalization_34 (Ba  (None, 19, 19, 16)        64        
 tchNormalization)                                               
                                                                 
 conv2d_35 (Conv2D)          (None, 19, 19, 16)        2320      
                                                                 
 batch_normalization_35 (Ba  (None, 19, 19, 16)       

In [30]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "/content/drive/Shareddrives/AICUP圍棋Dataset/save_PS_bestModel/best_model.h5",
    monitor = "val_loss",
    verbose = 0,
    save_best_only = True,
    save_weights_only = True,
    save_freq="epoch",
    options=None,
    initial_value_threshold=None,
)

In [31]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 128,
    epochs = 30,
    callbacks=[model_checkpoint_callback],
    validation_data=(x_val, y_val),
)

Epoch 1/30
188/188 [==============================] - 12s 19ms/step - loss: nan - accuracy: 0.3808 - val_loss: 1.1507 - val_accuracy: 0.3794
Epoch 2/30
188/188 [==============================] - 3s 16ms/step - loss: nan - accuracy: 0.4233 - val_loss: 1.1609 - val_accuracy: 0.3249
Epoch 3/30
188/188 [==============================] - 3s 13ms/step - loss: nan - accuracy: 0.4378 - val_loss: 1.0965 - val_accuracy: 0.4020
Epoch 4/30
188/188 [==============================] - 3s 13ms/step - loss: nan - accuracy: 0.4874 - val_loss: 1.0140 - val_accuracy: 0.5334
Epoch 5/30
188/188 [==============================] - 2s 13ms/step - loss: nan - accuracy: 0.5454 - val_loss: 0.9612 - val_accuracy: 0.5372
Epoch 6/30
188/188 [==============================] - 3s 14ms/step - loss: nan - accuracy: 0.5566 - val_loss: 0.9484 - val_accuracy: 0.5451
Epoch 7/30
188/188 [==============================] - 3s 17ms/step - loss: nan - accuracy: 0.5621 - val_loss: 0.9482 - val_accuracy: 0.5518
Epoch 8/30
188/188 

In [32]:
model.load_weights('/content/drive/Shareddrives/AICUP圍棋Dataset/save_PS_bestModel/best_model.h5')

# Testing

**PublicUpload.csv** must be in the following form:
```
PSTPU0000000001_79,1
PSTPU0000000001_7,1
PSTPU0000000001_153,1
PSTPU0000000001_115,1
PSTPU0000000001_131,1
PSTPU0000000001_99,1
PSTPU0000000001_21,2
```

- Column 1: Game ID
- Column 2: Predicted Game Style

The code block below is to use **play_style_test_public.csv** to predict and save the results in required form.

In [33]:
df = open('/content/drive/Shareddrives/AICUP圍棋Dataset/play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)
print(prediction_numbers)

313/313 [==============================] - 1s 3ms/step
[1 0 0 ... 2 1 2]


Save results to **PublicUpload.csv**.

In [34]:
with open('/content/drive/Shareddrives/AICUP圍棋Dataset/PublicUpload.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!